In [ ]:
import netCDF4
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib as plt
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [ ]:
# Opened the netCDF file and get the latitude and longitude variables
nc_file = netCDF4.Dataset('CSR_GRACE_GRACE-FO_RL06_Mascons_all-corrections_v02.nc')
lat_var = nc_file.variables['lat']
lon_var = nc_file.variables['lon']


In [ ]:
# Loaded the latitude and longitude values into numpy arrays
lat_values = lat_var[:]
lon_values = lon_var[:]

In [ ]:
# Loaded the selected latitude and longitude values from the dataframe
df = pd.read_csv('data.csv', header=None)
selected_lat = df[0][:].values
selected_lon = df[1][:].values
selected_lon

In [ ]:
selected_lon[selected_lon < 0] += 360 #Because the longitude is in -180-+180 format, this will help convert it to 0-360 of GRACE

# In summary, this line of code adds 360 to all negative values in the selected_lon array. 
# This is often done when working with longitude values in geographic data, 
# as it converts negative longitudes (i.e., those in the Western Hemisphere) to their 0-360 degree equivalent. 
# This can be useful when working with data that uses a 0-360 degree longitude range instead of the -180 to 180 degree range.

In [ ]:
# I Created a map plot with cartopy and matplotlib
fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

# Added coastlines and gridlines
ax.coastlines()
ax.gridlines(draw_labels=True, linestyle='--')

# Plotted the longitude and latitude points on the map
ax.scatter(selected_lon, selected_lat, color='red', marker='o', transform=ccrs.PlateCarree())

# Showed the plot
plt.show()

In [ ]:
# Found the indices of the latitude and longitude values that are closest to the selected values
lat_idx = np.abs(lat_values - selected_lat[:, np.newaxis]).argmin(axis=1)
lon_idx = np.abs(lon_values - selected_lon[:, np.newaxis]).argmin(axis=1)

In [ ]:
Newdf = pd.DataFrame([])#Created empty dataframe

In [ ]:
point_list = zip(selected_lat,selected_lon)

In [ ]:
ds = xr.open_dataset('CSR_GRACE_GRACE-FO_RL06_Mascons_all-corrections_v02.nc')

In [ ]:
# Defined the latitude and longitude index points
lat_indexes = lat_idx  # Replaced with  latitude index points
lon_indexes = lon_idx  # Replace with  longitude index points

# Created an empty DataFrame to store the extracted data
columns = ['lat', 'lon'] + [f'lwe_thickness_{i}' for i in range(0, 180)]
df = pd.DataFrame(columns=columns)

# Iterated over the specified lat and lon index pairs
for lat_idx, lon_idx in zip(lat_indexes, lon_indexes):
    lat = ds['lat'][lat_idx].values
    lon = ds['lon'][lon_idx].values

    # Extracted lwe_thickness for the specified indexes and lat-lon pair
    lwe_thickness_values = ds['lwe_thickness'][0:180, lat_idx, lon_idx].values

    # Stored the extracted data in the DataFrame
    data = {'lat': lat, 'lon': lon, **{f'lwe_thickness_{i}': val for i, val in zip(range(0, 180), lwe_thickness_values)}}
    df = df.append(data, ignore_index=True)

# Saved the DataFrame to a CSV file
df.to_csv('lwe_thickness.csv', index=False)

In [ ]:
time = nc_file.variables["time_bounds"][0:180,0:1]

In [ ]:
array = time.compressed()
array.size

In [ ]:
# defined a starting date and time
start_datetime = datetime(2002, 1, 1)

In [ ]:
# created an empty NumPy array to hold the resulting datetime objects
datetime_array = np.empty_like(array, dtype='datetime64[D]')

In [ ]:
# iterated over the array and add each value to the starting date
for i, value in enumerate(array):
    delta_days = int(value)
    time_datetime = start_datetime + timedelta(days=delta_days)
    datetime_array[i] = np.datetime64(time_datetime, 'D')

In [ ]:
datetime_array

In [ ]:
df.iloc[:, 2:].set_axis(datetime_array, axis='columns', inplace=True)#Replacing column name with the dates

In [ ]:
lwe_thickness_subset = df.iloc[:, 2:]#Subsetting from column 2-145(variables excluding lat-lon values)
lwe_thickness_subset.set_axis(datetime_array, axis ='columns', inplace=True)#Replacing column name with the dates
lat_lon_values = df.iloc[:, 0:2]

In [ ]:
lwe_thickness_subset
lat_lon_values

In [ ]:
lwe_thickness = pd.concat([lat_lon_values.iloc[:, 0:2], lwe_thickness_subset], axis=1)

In [ ]:
lwe_thickness

In [ ]:
lwe_thickness.to_csv('lwe_thickness_time_series.csv', index=False)#Exported to CSV